In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import scipy.stats

In [5]:
data = pd.read_csv('athleticnet_players.csv')
zcruit = pd.read_csv("ZCruit_Cleaned_Data.csv")

C:\Users\AlvinYu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,17,19,21,23,25,27,29,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
print("There are {} players with T&F data".format(data.shape[0]))
print("{}% of players in ZCruit database have T&F data".format(round(100 * (data.shape[0]/zcruit.shape[0])), 2))

There are 7672 players with T&F data
11% of players in ZCruit database have T&F data


In [13]:
data.head(5)

,Zcruit ID,Player Name,Grad Year,High School Name,High School City,High School State,4 x 100 Meter,4 x 100 Meter Event,4 x 100 Meter Occurred on,4 x 200 Meter,...,Javelin Occurred on,Discus,Discus Event,Discus Occurred on,Pole Vault,Pole Vault Event,Pole Vault Occurred on,Hammer,Hammer Event,Hammer Occurred on
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,2021,Birdville High School,Haltom City,TX,43.29s,UIL 5A District 07 (W-Th) Track Meet,4/3/2019,98.08s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,2022,Denison Senior High School,Denison,TX,44.42s,Liberty Christian Bubba Joiner Invitational (F...,3/5/2020,96.86s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,2022,Southern Regional 9-10 High School,Manahawkin,NJ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,2021,Santa Monica High School,Santa Monica,CA,43.58s,Mira Costa vs Santa Monica Dual Meet,2/27/2020,96.31s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,2021,Vaca High School,Vacaville,CA,45.02s,Stocking Super Seven Invitational,2/29/2020,97.20s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
tf_metrics = ["100 Meter", "200 Meter", "400 Meter", 
              "110 Meter Hurdles", "300 Meter Hurdles", 
              "High Jump", "Long Jump", "Triple Jump",
              "Shotput", "Discus", "Hammer"]
tf_metrics_w_player_info = ["Zcruit ID", "Player Name"] + tf_metrics

In [19]:
tf_data = data[tf_metrics_w_player_info]

In [20]:
tf_data

,Zcruit ID,Player Name,100 Meter,200 Meter,400 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus,Hammer
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,11.25s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,NaN,24.95s,60.38s,NaN,NaN,"5' 6""","18' 1""",NaN,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,NaN,25.90s,NaN,NaN,NaN,NaN,NaN,NaN,"40' 10.5""",NaN,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,12.51 s,24.94s,64.33s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,11.74s,25.77 s,NaN,NaN,47.84s,NaN,"17' 3.5""",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667,f1873c0b-a931-4604-a33c-4439e32789eb,Jerod Thomas,12.26s,NaN,NaN,NaN,NaN,"5' 4""",NaN,NaN,NaN,NaN,NaN
7668,1f7b327a-510e-4070-a3cd-bc0d2433010f,Tyler Bradley,11.51s,23.05 s,NaN,NaN,NaN,NaN,"16' 7.25""","36' 1.5""",NaN,NaN,NaN
7669,6bcf9f7b-4c2d-4db2-a19e-bdea9c358d04,Jesse James Garcia,12.23s,24.16s,NaN,NaN,NaN,NaN,"14' 8.5""",NaN,NaN,NaN,NaN
7670,fb194f5e-bc21-4852-838b-443eebac9aaf,Isaiah Robinson,11.53s,24.10s,NaN,NaN,NaN,NaN,"19' 6""",NaN,NaN,NaN,NaN


In [21]:
def nan_counter(data, metrics):
    nan_dict, readable_dict = {}, {}
    for metric in metrics:
        nan_dict[metric] = data[metric].isna().sum() / data.shape[0]
        readable_dict[metric] = "There are {} NaNs out of {} players".format(data[metric].isna().sum(), data.shape[0])
    print(readable_dict)
    return nan_dict

In [22]:
nan_counter(tf_data, tf_metrics)

{'100 Meter': 'There are 3544 NaNs out of 7672 players', '200 Meter': 'There are 3561 NaNs out of 7672 players', '400 Meter': 'There are 5652 NaNs out of 7672 players', '110 Meter Hurdles': 'There are 7064 NaNs out of 7672 players', '300 Meter Hurdles': 'There are 7042 NaNs out of 7672 players', 'High Jump': 'There are 6693 NaNs out of 7672 players', 'Long Jump': 'There are 5204 NaNs out of 7672 players', 'Triple Jump': 'There are 6634 NaNs out of 7672 players', 'Shotput': 'There are 5387 NaNs out of 7672 players', 'Discus': 'There are 5893 NaNs out of 7672 players', 'Hammer': 'There are 7665 NaNs out of 7672 players'}


{'100 Meter': 0.4619395203336809,
 '200 Meter': 0.464155370177268,
 '400 Meter': 0.7367049009384776,
 '110 Meter Hurdles': 0.9207507820646507,
 '300 Meter Hurdles': 0.9178832116788321,
 'High Jump': 0.872393117831074,
 'Long Jump': 0.678310740354536,
 'Triple Jump': 0.8647028154327424,
 'Shotput': 0.7021637122002086,
 'Discus': 0.7681178310740354,
 'Hammer': 0.9990875912408759}

We can throw out hammer since 99.9% of players don't have a Hammer throw.

In [32]:
pandas.Series.str.replace(' ', '')
"^(\d)'\s*(\d*\.*\d*)"
def height_str_to_val(height_str):
    match = re.match("^(\d)'\s*(\d*\.*\d*)", height_str)
    groups = match.groups()
    feet, inches = int(groups[0]), float(groups[1])
    return feet * 12 + inches 

ValueError: could not convert string to float: '25. 77'

In [39]:
tf_data['100 Meter'].str.rstrip('s(.0)(.1)(.2)(.3)(.4)(.5)(.6)(.7)(.8)(.9)').astype(float)

ValueError: could not convert string to float: ''

In [44]:
pd.set_option('display.max_rows', 200)
tf_data[tf_data['100 Meter'].notna()].head(200)

,Zcruit ID,Player Name,100 Meter,200 Meter,400 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus,Hammer
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,11.25s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,12.51 s,24.94s,64.33s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,11.74s,25.77 s,NaN,NaN,47.84s,NaN,"17' 3.5""",NaN,NaN,NaN,NaN
5,c98e0321-f3a4-4b85-ab2a-bbd16d46ea86,Isaac Jones,11.73s,23.28s,NaN,NaN,NaN,NaN,"16' 2.5""",NaN,NaN,NaN,NaN
6,7e546367-e3bd-44b4-a1e7-1b2cf182d224,David Godsey,11.17s,23.18s,54.17s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20396bae-d68b-4b18-858e-e341d20f2b85,Hunt Young,12.22s,24.68s,55.13s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,a6b7e3c4-832d-4f6d-a039-e2af26012e82,Jahbez Hawkins,11.24s,23.86s,NaN,NaN,NaN,"5' 8""",NaN,NaN,NaN,NaN,NaN
10,3df5eead-7148-452e-a38d-8adf743013a1,D'Angelo Stocker,10.99s,22.94s,54.25s,NaN,NaN,NaN,"16' 0.5""",NaN,NaN,NaN,NaN
11,b5c9b532-5606-474b-ae57-3431fed280bf,Cole Joyce,11.35 s,23.16 s,NaN,NaN,NaN,NaN,NaN,NaN,"42' 11.5""",NaN,NaN
12,16002f03-bfe1-49d8-b7b8-7ab827c86399,Fitzroy Ledgister,11.64s,24.57s,56.51s,18.07s,NaN,"5' 6""","21' 5.25""",NaN,"35' 7""",NaN,NaN


In [ ]:
.